# Exercises

# Data Acquisition

# These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

# This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

In [50]:
#### Imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as X
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# 1. Read the case, department, and source data into their own spark dataframes.

In [4]:
df_case = spark.read.csv("case.csv", sep = ",", header = True, inferSchema = True)
df_case.show(3, vertical = True, truncate = False)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [5]:
df_dept = spark.read.csv("dept.csv", sep = ",", header = True, inferSchema = True)
df_dept.show(5, vertical = True, truncate = False)

-RECORD 0-----------------------------------------------
 dept_division          | 311 Call Center               
 dept_name              | Customer Service              
 standardized_dept_name | Customer Service              
 dept_subject_to_SLA    | YES                           
-RECORD 1-----------------------------------------------
 dept_division          | Brush                         
 dept_name              | Solid Waste Management        
 standardized_dept_name | Solid Waste                   
 dept_subject_to_SLA    | YES                           
-RECORD 2-----------------------------------------------
 dept_division          | Clean and Green               
 dept_name              | Parks and Recreation          
 standardized_dept_name | Parks & Recreation            
 dept_subject_to_SLA    | YES                           
-RECORD 3-----------------------------------------------
 dept_division          | Clean and Green Natural Areas 
 dept_name              | Parks

In [6]:
df_source = spark.read.csv("source.csv", sep = ",", header = True, inferSchema = True)
df_source.show(5, truncate = False)

+---------+----------------+
|source_id|source_username |
+---------+----------------+
|100137   |Merlene Blodgett|
|103582   |Carmen Cura     |
|106463   |Richard Sanchez |
|119403   |Betty De Hoyos  |
|119555   |Socorro Quiara  |
+---------+----------------+
only showing top 5 rows



# 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [7]:
df_source.write.csv("df_source_csv", mode = "overwrite")
df_source.write.json("df_source_json", mode = "overwrite")

- Inspect your folder structure. What do you notice?

# 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [8]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [9]:
df_case.explain()

== Physical Plan ==
FileScan csv [case_id#16,case_opened_date#17,case_closed_date#18,SLA_due_date#19,case_late#20,num_days_late#21,case_closed#22,dept_division#23,service_request_type#24,SLA_days#25,case_status#26,source_id#27,request_address#28,council_district#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/rajaramgautam/codeup-data-science/spark-exercises/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...




In [10]:
df_dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [11]:
df_source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [12]:
# Joining the data frames:
df = df_case.join(df_dept, 'dept_division', 'left').join(df_source, 'source_id', 'left')

In [13]:
df_case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [14]:
# Comparing 'case' df above to the joined df below:
df.show(2, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 1/1/18 0:42                          
 case_closed_date       | 1/1/18 12:29                         
 SLA_due_date           | 9/26/20 0:42                         
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [24]:
# Changing date to datetime
df = (
    df.withColumn('case_opened_date', X.to_timestamp(X.col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', X.to_timestamp(X.col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('SLA_due_date', X.to_timestamp(X.col('SLA_due_date'), 'M/d/yy H:mm'))
)

In [25]:
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



# 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [27]:
(
    df.select(X.datediff(X.current_timestamp(), 'SLA_due_date')
    .alias('days_past_due'))
    .where(df.case_closed == 'NO')
    .sort(X.col('days_past_due').desc())
    .show(3)
)

+-------------+
|days_past_due|
+-------------+
|         1856|
|         1856|
|         1856|
+-------------+
only showing top 3 rows



# 2. How many Stray Animal cases are there?

In [28]:
(
    df.filter(df.service_request_type == 'Stray Animal')
    .count()
)

27361

# 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [29]:
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

# 4. Convert the council_district column to a string column.

In [30]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [32]:
# Converting to string type:
df = df.withColumn("council_district", X.col("council_district").cast("string"))

In [33]:
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



# 5. Extract the year from the case_closed_date column.

In [35]:
df.select(df.case_closed_date, X.year(df.case_closed_date)).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



# 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [36]:
df.select(df.num_days_late, (df.num_days_late * 24).alias('num_of_hours_late')).show(5)

+-------------------+-------------------+
|      num_days_late|  num_of_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



# 7. Join the case data with the source and department data.

In [37]:
# Completed already
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



# 8. Are there any cases that do not have a request source?

In [38]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [40]:
df.filter(X.isnull(df.source_id)).count()

0

# 9. What are the top 10 service request types in terms of number of requests?

In [42]:
(
    df.select(df.service_request_type)
    .groupBy(df.service_request_type)
    .count().sort(X.col('count').desc()).show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



# 10. What are the top 10 service request types in terms of average days late?

In [44]:
(
    df.where(df.case_late == 'YES') 
    .groupBy(df.service_request_type)
    .agg(X.mean(df.num_days_late).alias('n_days_late'), X.count('*').alias('n_cases'))
    .sort(X.desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



# 11. Does number of days late depend on department?

# 12. How do number of days late depend on department and request type?

In [48]:
(
    df.groupBy(df.dept_division).agg(X.avg(df.num_days_late).alias('days_late')).sort(X.desc('days_late')).show()
)

+--------------------+-------------------+
|       dept_division|          days_late|
+--------------------+-------------------+
|Code Enforcement ...| 135.92851612479797|
|        Reservations|        66.03116319|
|     311 Call Center|  59.73709149630077|
|Director's Office...| 37.570646702950064|
|Engineering Division| 13.433724555869714|
|               Shops|  9.641261768722687|
|           Tree Crew|    4.7232828120654|
|         Solid Waste| 3.5190239198762248|
|              Trades| 3.2319771412769334|
|Clean and Green N...|  1.691468919487805|
|              Vector|-1.1206532993223441|
|    Facility License| -1.412693770221664|
|       Miscellaneous|-1.7218576838926671|
|    Waste Collection| -2.170652238479902|
|     Clean and Green| -2.557154979254144|
|               Brush|-3.9857905714570996|
| Food Establishments| -6.971552370451528|
|  Signs and Markings| -7.448628001357722|
|    Shops (Internal)|  -8.18626711987645|
|         Storm Water|-14.055678397031896|
+----------